We should've used classes rather than functions to define our architecture.

The residual connections were also missed before -- both improvements below

In [1]:
from data import *
from model import *

import torch
import numpy as np
import pandas as pd
import os
import h5py
from exabiome.nn.loader import read_dataset, LazySeqDataset, train_test_loaders
import argparse
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from typing import Type, Any, Callable, Union, List, Optional

In [161]:
!nvidia-smi

Sat Jun 19 15:00:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:1A:00.0 Off |                    0 |
| N/A   34C    P0    64W / 300W |  13956MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
hparams = argparse.Namespace(**{'load': False,
                            'window': 4096,
                            'step': 4096,
                             'classify': False,
                               'tgt_tax_lvl': "species",
                               'fwd_only': True,
                               'manifold': True})

In [3]:
path = '/global/homes/a/azaidi/ar122_r202.toy.input.h5'
chunks = LazySeqDataset(hparams, path=path,
                       keep_open=True)

In [32]:
dl0, dl1, dl2 = train_test_loaders(chunks, batch_size=16, distances=True)
#batch = next(iter(dl2))
#train_test_loaders??

In [4]:
def get_toy_dl(hparams, batch_size=16):
    path = '/global/homes/a/azaidi/ar122_r202.toy.input.h5'
    chunks = LazySeqDataset(hparams, path=path,
                           keep_open=True)
    ds = taxon_ds(chunks, old_pad_seq)
    return DataLoader(ds, batch_size=batch_size, 
                      shuffle=True), ds

In [6]:
dl,ds = get_toy_dl(hparams, batch_size=8)
batch = next(iter(dl))
len(dl)
, batch[0].shape, batch[1].shape

(2377, torch.Size([8, 1, 4096]), torch.Size([8]))

Let's creat a block class that we'll use for the inverted residual blocks

In [7]:
class DepSepBlock(nn.Module):
    def __init__(self):
        super(DepSepBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, 3)
        self.bn1 = nn.BatchNorm1d(32)
        self.act = nn.SiLU(inplace=True)
        self.conv2 = nn.Conv1d(32, 8, 1)
        self.conv3 = nn.Conv1d(8, 32, 1)
        self.conv4 = nn.Conv1d(32, 16, 1, stride=1)
        self.bn2 = nn.BatchNorm1d(16)
    
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act(out)
        out = self.conv2(out)
        out = self.act(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.bn2(out)
        #out = nn.Identity(out)
        
        return out

In [8]:
dsc_out = nn.Conv1d(1, 32, kernel_size=3, stride=2, padding=1)(batch[0])
dsc_out.shape

torch.Size([8, 32, 2048])

In [9]:
DepSepBlock()(dsc_out).shape

torch.Size([8, 16, 2046])

In [102]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_ch:int, mid_ch:int, sq_ch:int,
                 out_ch:int, ks=1, stride: int = 1, padding: int =0):
        super(InvertedResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv1d(in_ch, mid_ch, kernel_size=1, stride=1)
        self.bn1 = nn.BatchNorm1d(mid_ch)
        self.act = nn.SiLU(inplace=True)
        self.conv2 = nn.Conv1d(mid_ch, mid_ch, kernel_size=ks, stride=stride,
                              padding=padding)
        self.bn2 = nn.BatchNorm1d(mid_ch)
        #self.conv3 = nn.Conv1d(mid_ch, sq_ch, kernel_size=1,
        #                      stride=1) #squeeze excite conv
        #self.conv4 = nn.Conv1d(sq_ch, mid_ch, kernel_size=1,
        #                      stride=1) #squeeze excite conv
        self.squeeze = SqueezeExcite(mid_ch, sq_ch)
        self.conv5 = nn.Conv1d(mid_ch, out_ch, kernel_size=1, stride=1) 
        self.bn3 = nn.BatchNorm1d(out_ch)
        
        self.downsample = nn.Conv1d(in_ch, out_ch,1, stride=stride)
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act(out)
        out = self.conv2(out)
        out = self.bn2(out)
        #out = self.conv3(out)
        #out = self.act(out)
        #out = self.conv4(out)
        out = self.squeeze(out)
        out = self.conv5(out)
        out = self.bn3(out)
        
        identity = self.downsample(x)
        out += identity
        out = self.act(out)
        
        return out

In [53]:
irb = InvertedResidualBlock(1, 5, 3, 6, 5, 2,2)
irb

InvertedResidualBlock(
  (conv1): Conv1d(1, 5, kernel_size=(1,), stride=(1,))
  (bn1): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
  (conv2): Conv1d(5, 5, kernel_size=(5,), stride=(2,), padding=(2,))
  (bn2): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (squeeze): SqueezeExcite(
    (layer): Sequential(
      (0): Conv1d(5, 3, kernel_size=(1,), stride=(1,))
      (1): SiLU(inplace=True)
      (2): Conv1d(3, 5, kernel_size=(1,), stride=(1,))
    )
  )
  (conv5): Conv1d(5, 6, kernel_size=(1,), stride=(1,))
  (bn3): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (downsample): Conv1d(1, 6, kernel_size=(1,), stride=(2,))
)

In [54]:
irb(batch[0]).shape

torch.Size([8, 6, 2048])

Using the class below makes our model description look terrible -- so we'll keep this out of the model definition

In [111]:
class ConvBnAct(nn.Module):
    def __init__(self, in_ch, out_ch, ks=1, stride=1, padding=0, activation=False):
        super().__init__()
        layer_list = []
        layer_list.append(nn.Conv1d(in_ch, out_ch,
                                   kernel_size=ks, stride=stride,
                                   padding=padding))
        layer_list.append(nn.BatchNorm1d(out_ch))
        if activation:
            layer_list.append(nn.SiLU(inplace=True))
        self.layer_list = nn.Sequential(*layer_list)
    def forward(self, x):
        out = self.layer_list(x)
        return out + x

In [109]:
ConvBnAct(1, 5, activation=True)#(batch[0])

ConvBnAct(
  (layer_list): Sequential(
    (0): Conv1d(1, 5, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU(inplace=True)
  )
)

In [112]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_ch:int, mid_ch:int, sq_ch:int,
                 out_ch:int, ks=1, stride: int = 1, padding: int =0):
        super(InvertedResidualBlock, self).__init__()
        
        self.conv1 = ConvBnAct(in_ch, mid_ch, activation=True)
        self.conv2 = ConvBnAct(mid_ch, mid_ch, ks=ks, stride=stride, padding=padding)
        #self.squeeze = SqueezeExcite(mid_ch, sq_ch)
        self.conv3 = ConvBnAct(mid_ch, out_ch)

        self.downsample = nn.Conv1d(in_ch, out_ch,1, stride=stride)
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.squeeze(out)
        out = self.conv3(out)
        
        identity = self.downsample(x)
        out += identity
        
        return out

In [114]:
#uncomment to confirm this does not look like what we want
#InvertedResidualBlock(1, 6, 8, 20)#(batch[0])

We can wrap the squeeze excite up into it's own class though, similar to how it's done in Timm

In [55]:
class SqueezeExcite(nn.Module):
    def __init__(self, ch_in, mid_ch):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv1d(ch_in, mid_ch, kernel_size=1, stride=1),
            nn.SiLU(inplace=True),
            nn.Conv1d(mid_ch, ch_in, kernel_size=1, stride=1)
        )
    def forward(self, x):
        out = self.layer(x)
        return out + x

Here's our final version of the inverted residual block

In [119]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_ch:int, mid_ch:int, sq_ch:int,
                 out_ch:int, ks=1, stride: int = 1, padding: int =0):
        super(InvertedResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv1d(in_ch, mid_ch, kernel_size=1, stride=1)
        self.bn1 = nn.BatchNorm1d(mid_ch)
        self.act = nn.SiLU(inplace=True)
        self.conv2 = nn.Conv1d(mid_ch, mid_ch, kernel_size=ks, stride=stride,
                              padding=padding)
        self.bn2 = nn.BatchNorm1d(mid_ch)
        self.squeeze = SqueezeExcite(mid_ch, sq_ch)
        self.conv5 = nn.Conv1d(mid_ch, out_ch, kernel_size=1, stride=1) 
        self.bn3 = nn.BatchNorm1d(out_ch)
        
        self.downsample = nn.Conv1d(in_ch, out_ch,1, stride=stride)
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.act(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.squeeze(out)
        out = self.conv5(out)
        out = self.bn3(out)
        
        identity = self.downsample(x)
        out += identity
        out = self.act(out)
        
        return out

Let's also incoroprate a make layer function that will 

In [120]:
def make_layer(param_list):
    layers = []
    for x in range(len(param_list)):
        layers.append(InvertedResidualBlock(*param_list[x]))
    return nn.Sequential(*layers)

In [121]:
ll = []
for x in range(len(p_list)):
    ll.append(make_layer(param_list = p_list[0]))

In [123]:
#uncomment below to confirm this is doing what is desired
#ll

Previously our model definition had us explicitly calling each layer group and then having to pass them one by one in the forward implementation. With the layer list function above, we can clean up our model definition!

In [156]:
class TestNet(nn.Module):
    def __init__(self, param_list, avg_out=200, out_feats=512):
        super(TestNet, self).__init__()

        self.avg_out = 18
        self.out_feats = out_feats
        self.param_list = param_list

        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, stride=2, padding=1)
        self.bn1 = nn.BatchNorm1d(32)
        self.silu = nn.SiLU(inplace=True)
        self.layer0 = DepSepBlock()
        self.layer_list = []
        for x in range(len(p_list)):
            self.layer_list.append(self._make_layer(self.param_list[x]))
        self.layer_list = nn.Sequential(*self.layer_list)
        self.conv2 = nn.Conv1d(320, 18, 1, 1)
        self.bn2 = nn.BatchNorm1d(18)
        self.avgpool = nn.AdaptiveAvgPool1d(output_size=self.avg_out)
        self.fc = nn.Linear(in_features=self.avg_out, 
                            out_features=self.out_feats)
    
    def _make_layer(self, param_list):
        layers = []
        for x in range(len(param_list)):
            layers.append(InvertedResidualBlock(*param_list[x]))
        return nn.Sequential(*layers)
        
    def _forward_impl(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.silu(x)
        
        x = self.layer0(x)
        x = self.layer_list(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.avgpool(x)
        x = self.fc(x)
        return x
    
    def forward(self, x):
        return self._forward_impl(x)

Unfortunately, we are still hardcoding the values into our model definition, w/e

In [157]:
p_list = [
        [[16,96,3,24,3,2,1], [24,144,6,24,3,1,1]], #layer 1
        [[24,144,6,40,5,2,2], [40,240,10,40,5,1,2]], #layer 2 
        [[40,240,10,80,3,2,1],[80,480,20,80,3,1,1], 
                                [80,480,20,80,3,1,1]], #layer 3
        [[80,480,20,112,5,1,2], [112,672,28,112,5,1,2], 
                                 [112,672,28,112,5,1,2]], #layer 4
        [[112,672,28,192,5,2,2],[192,1152,48,192,5,1,2],
         [192,1152,48,192,5,1,2], [192,1152,48,192,5,1,2]], #layer 5
        [[192,1152,48,320,3,2,1]] #layer 6
         ]

In [158]:
m = TestNet(p_list, out_feats=1, avg_out=18)
#m
m(batch[0]).shape

torch.Size([8, 18, 1])

In [144]:
out = TestNet(p_list)(batch[0])
out.shape

torch.Size([8, 1280, 512])

The above cell confirms that our model can complete a forward pass -- the below cell shows our previous (disgusting) definition of the model

In [136]:
def get_eff_b0(out_feats):
    return nn.Sequential(
        get_base_layer(),
        get_dep_sep(32, 16),

        #layer 1 has two inverted residuals (IRs)
        get_inv_res(in_ch=16, mid_ch=96, out_ch=24,
                   sq_ch=4, ks=3, stride=2, padding=1),
        get_inv_res(in_ch=24, mid_ch=144, out_ch=24,
                   sq_ch=6, ks=3, stride=1, padding=1),

        #layers 2 has 2 IR's
        get_inv_res(in_ch=24, mid_ch=144, out_ch=40,
                   sq_ch=6, ks=5, stride=2, padding=2),
        get_inv_res(in_ch=40, mid_ch=240, out_ch=40,
                   sq_ch=10, ks=5, stride=2, padding=2),

        #layer 3 has 3 IR's
        get_inv_res(in_ch=40, mid_ch=240, out_ch=80,
                   sq_ch=10, ks=3, stride=2, padding=1),
        get_inv_res(in_ch=80, mid_ch=480, out_ch=80,
                   sq_ch=20, ks=3, stride=1, padding=1),
        get_inv_res(in_ch=80, mid_ch=480, out_ch=80,
                   sq_ch=20, ks=3, stride=1, padding=1),

        #layer 4 has 3 inverted residuals
        get_inv_res(in_ch=80, mid_ch=480, out_ch=112,
                   sq_ch=20, ks=5, stride=1, padding=2),
        get_inv_res(in_ch=112, mid_ch=672, out_ch=112,
                   sq_ch=28, ks=5, stride=1, padding=2),
        get_inv_res(in_ch=112, mid_ch=672, out_ch=112,
                   sq_ch=28, ks=5, stride=1, padding=2),

        #layer 5 has 4 inverted residuals
        get_inv_res(in_ch=112, mid_ch=672, out_ch=192,
                   sq_ch=28, ks=5, stride=2, padding=2),
        get_inv_res(in_ch=192, mid_ch=1152, out_ch=192,
                   sq_ch=48, ks=5, stride=1, padding=2),
        get_inv_res(in_ch=192, mid_ch=1152, out_ch=192,
                   sq_ch=48, ks=5, stride=1, padding=2),
        get_inv_res(in_ch=192, mid_ch=1152, out_ch=192,
                   sq_ch=48, ks=5, stride=1, padding=2),
        #layer 6 has 1 IR
        get_inv_res(in_ch=192, mid_ch=1152, out_ch=320,
                   sq_ch=48, ks=3, stride=1, padding=1),

        get_head_layer(out_chans=out_feats)
    )

In [160]:
get_eff_b0(18)(batch[0]).shape

torch.Size([8, 18, 1])

In [30]:
effnet_b0

Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv1d(1, 32, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): SiLU(inplace=True)
  )
  (1): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,), bias=False)
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv1d(32, 8, kernel_size=(2,), stride=(2,))
      (1): SiLU()
      (2): Conv1d(8, 32, kernel_size=(2,), stride=(2,))
    )
    (2): Sequential(
      (0): Conv1d(32, 16, kernel_size=(1,), stride=(1,), bias=False)
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Identity()
  )
  (2): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Conv1d(16, 96, kernel_